In [1]:
import io
import os
import re
from xml.dom import minidom
import json
import codecs

In [5]:
#f = io.open("igt_cat.json", mode="r", encoding="utf-8")
#cwd = os.getcwd()
# Print the current working directory
#print("Current working directory: {0}".format(cwd))
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))
# f.read()
os.listdir(cwd)
res = []
# Iterate directory
for path in os.listdir(cwd):
    # check if current path is a file
    if os.path.isfile(os.path.join(cwd, path)):
        res.append(path)
print(len(res))

Current working directory: C:\Users\nlutt\Documents\Websites\ckCollections\recipes_xml\03 Mangold
882


Find out for single file name.

In [3]:
s = "recipe-1021541207378794 - Ananas - Erdbeer - Konfitüre.xml"
id = re.match(r"recipe-(\d+)", s).group(1)
print (id)

1021541207378794


In [4]:
rn = re.match(r"recipe-\d+\s-\s(.+)\.xml",s).group(1)
rn = re.sub(r"\s-\s", '-' ,rn)
print (rn)

Ananas-Erdbeer-Konfitüre


In [5]:
fn_new = rn + ' - r' + id + '.xml'
print (fn_new)

Ananas-Erdbeer-Konfitüre - r1021541207378794.xml


In [7]:
# Iterate directory
#for fn in os.listdir(cwd):
#    id = re.match(r"recipe-(\d+)", fn).group(1)
#    rn = re.match(r"recipe-\d+\s-\s(.+)\.xml", fn).group(1)
#    rn = re.sub(r"\s-\s", '-' , rn)
#    fn_new = rn + ' - r' + id + '.xml'
#    os.rename (fn, fn_new)

In [25]:
clean = os.listdir(cwd)
print (len(clean))

882


In [ ]:
# Iterate directory
#for fn in os.listdir(cwd):
#    fn_new = re.match(r"(.*)\s-\sr", fn).group(1) + '.xml'
#    os.rename (fn, fn_new)
print (len(os.listdir(cwd)))

In [8]:
with open('C:/Users/nlutt\Documents/Websites/Ingredients/igt-catalog.json', 'r', encoding='utf-8') as f:
  igtCat = json.load(f)
cat = igtCat.get('ingredients')
print (len(cat))
#for igt in cat:
#    print (igt.get('normName'))

710


In [22]:
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
fn = "Schalottenpfannekuchen mit Ziegenkäse gefüllt.xml"
recipe = minidom.parse (fn)
ingredients = recipe.getElementsByTagName("fr:igdtName")
rc = []
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
getIngredientNames (ingredients)
print (rc)

['Schalotte(n)', 'Olivenöl', 'Mehl', 'Milch', 'Ei(er)', 'Butter, flüssige', 'Mangold', 'Lauch', 'Käse (schnittfester Ziegenkäse)', 'Maiskörner, vorgegarte (Dose)', 'Knoblauch', 'Petersilie, fein geschnittene', 'Salz und Pfeffer aus der Mühle', 'Cayennepfeffer', 'Muskat, frisch gerieben']


Read and parse ingredient catalogue.

In [27]:
#middle loop: recipe ingredient name fields
for igt_in_rcp in rc:
    given = igt_in_rcp.lower()
    oder = re.match(r"(.*)\soder\s",given)
    if oder != None:
        given = oder.group(1)
    matches = []
    myMatch = {}
    #inner loop: normName and synonymes
    for igt in cat:
        x = igt.get('normName')
        m = re.search (x, given)
        if m != None:
            start, end = m.span()
            if end-start > 0:
                #print (end-start, '  ', m.group() )
                matching = {}
                matching['name']   = igt.get('id')
                matching['pos']    = start
                matching['length'] = end - start
                matches.append(matching) 
        synos = igt.get('synonyms')
        if synos != None:
            for syno in synos:
                m = re.search (syno, given)
                if m != None:
                    start, end = m.span()
                    if end-start > 0:
                        #print (end-start, '  ', m.group() )
                        matching = {}
                        matching['name']   = igt.get('id')
                        matching['pos']    = start
                        matching['length'] = end - start
                        matches.append(matching)
    if len(matches) > 0:
        matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
        myMatch = matches_sorted[0]
        print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
    if myMatch == {}:
        no_match_ctr += 1
#        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
        print (message)

     +  schalotte(n)                                 schalotten
     +  olivenöl                                     olivenöl
     +  mehl                                         weizenmehl
     +  milch                                        milch
     +  ei(er)                                       ei
     +  butter, flüssige                             butter
     +  mangold                                      mangold
     +  lauch                                        lauch
     +  käse (schnittfester ziegenkäse)              ziegenkäse
     +  maiskörner, vorgegarte (dose)                mais
     +  knoblauch                                    knoblauch
     +  petersilie, fein geschnittene                petersilie
     +  salz und pfeffer aus der mühle               
     +  cayennepfeffer                               cayennepfeffer
     +  muskat, frisch gerieben                      muskat


Next steps:
* change recipe name in recipe file to file name
* iterate over all recipes in collection
* insert normalized name into recipe ingredients list

Exchange recipe name

In [12]:
fn_noext = fn[0:fn.index(".xml")]
print (fn_noext) 
rn = recipe.getElementsByTagName("fr:recipeName")[0].firstChild
print (rn.nodeValue)

Alex vegane Mangold-Kokosmilch-Suppe
recipe-2876671439968626 - Alex vegane Mangold-Kokosmilch-Suppe


New recipe name

In [13]:
rn.replaceWholeText(fn_noext)
print(recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue)

Alex vegane Mangold-Kokosmilch-Suppe


Write to recipe file.

In [29]:
rcp_str = str(recipe.toxml())
with codecs.open('xx.xml', 'w', encoding = 'utf8') as file:
  file.write(rcp_str)
    

Design outer loop

In [9]:
def getIngredientNames(ingredients):
    for ingredient in ingredients:
        xx = ingredient.childNodes
        for x in xx:
            if x.nodeType == x.TEXT_NODE:
                rc.append(x.data)
                
i = 0
no_match_ctr = 0
os.chdir('C:/Users/nlutt/Documents/Websites/ckCollections/recipes_xml/03 Mangold')
cwd = os.getcwd()
for fn in os.listdir(cwd):
    print (fn)
    recipe = minidom.parse (fn)
    ingredients = recipe.getElementsByTagName("fr:igdtName")
    rc = []
    getIngredientNames (ingredients)
    i += 1
    #middle loop: recipe ingredient name fields
    for igt_in_rcp in rc:
        given = igt_in_rcp.lower()
        oder = re.match(r"(.*)\soder\s",given)
        if oder != None:
            given = oder.group(1)
        matches = []
        myMatch = {}
        #inner loop: normName and synonymes
        for igt in cat:
            x = igt.get('normName')
            m = re.search (x, given)
            if m != None:
                start, end = m.span()
                if end-start > 0:
                    #print (end-start, '  ', m.group() )
                    matching = {}
                    matching['name']   = igt.get('id')
                    matching['pos']    = start
                    matching['length'] = end - start
                    matches.append(matching) 
            synos = igt.get('synonyms')
            if synos != None:
                for syno in synos:
                    m = re.search (syno, given)
                    if m != None:
                        start, end = m.span()
                        if end-start > 0:
                            #print (end-start, '  ', m.group() )
                            matching = {}
                            matching['name']   = igt.get('id')
                            matching['pos']    = start
                            matching['length'] = end - start
                            matches.append(matching)
        if len(matches) > 0:
            matches_sorted = sorted(matches, key=lambda x: (-x['length'], x['pos']))
            myMatch = matches_sorted[0]
            print ('+'.rjust(6) + '  ' + given.ljust(45) + myMatch['name'])   
        if myMatch == {}:
            no_match_ctr += 1
    #        message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(35) + recipe.getElementsByTagName("fr:recipeName")[0].firstChild.nodeValue
            message = str(no_match_ctr).rjust(6) + '  ' + given.ljust(60) + fn
            print (message)

Adas bil Hamod.xml
     +  linsen, braune                               linsen
     +  wasser,                                      
     +  mangold                                      mangold
     +  zwiebel(n)                                   zwiebel
     +  olivenöl                                     olivenöl
     +  kreuzkümmel, gemahlener (cumin)              cumin
     +  minze, getrocknete                           minze
     +  zitrone(n), den saft                         zitrone
     +  salz, nach bedarf                            
     +  mehl                                         weizenmehl
Albertos Mangold-Ravioli mit Chickenwings und Shiitake-Pilze.xml
     +  mehl, 405                                    weizenmehl
     +  eigelb                                       ei
     +  ei(er)                                       ei
     +  kurkuma                                      curcuma
     +  olivenöl                                     olivenöl
     +  salz          